In [1]:
# 將 training data 的 pickle 檔載入
import pickle
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
import monpa as mp
import numpy as np
from tqdm import tqdm #進度條
mp.use_gpu(True)

train_df = pickle.load(open("./monpa_train.pickle", "rb"))
lables = list(train_df["Lable"])
contents = list(train_df["Content"])
print(contents[1])
le = preprocessing.LabelEncoder()
le.fit(["tech", "edu", "sports", "health", "travel", "politics"])
encoded_lable = le.transform(lables)




+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.
GPU unavailable.
披值 星帶 成 歷史   國軍 7 / 7 起 取消     軍 中 掛值 星帶 操課 將 成為 歷史 ， 國防部 考量 ， 帶值 星帶 對 野外 操課 造成 不便 ， 因此 在 這 個 月 7 日 ， 下令 三軍 部隊 取消 這 項 存在 超過 50 年 的 規定 。 不過 ， 外界 反應 不一 ， 曾 當 過 班長 帶兵 的 國民黨 立委 徐耀昌 就 表示 反對 ， 但 也 有 立委 相當 贊同 。 軍教片 當紅 ， 勾起 許多 人 當兵 的 回憶 ， 尤其 是 披值 星帶 的 教育 班長 ，   最 叫 人 印象 深刻 ， 不過 軍方 經過 8 個 月 的 研議 ，   在 7月 7日 ， 下令 三軍 部隊 ， 取消 披戴值 星帶 。 國


In [2]:
tg=['Dfa','Dfb','Nv','VA','VAC','VB','VC','VCL','VD','VE','VF','VG','VH','VHC','VI','VJ','VK','VL','Na','A','Nc','Ncd','ORG','PER']
contents2=[]

for content in tqdm(contents):
    chosen_words=[]
    
    psegs=mp.pseg_batch(content)[0]
    for pseg in psegs:
        try:
            if pseg[1] in tg:
                chosen_words.append(pseg[0].strip())
        except:
            print(pseg)
    #print(chosen_words)
    contents2.append(" ".join(chosen_words))

"""
for content in tqdm(contents):
    chosen_words=[]
    for word in content.split():
        #print(word)
        psegs=mp.pseg_batch(word)[0]
        for pseg in psegs:
            try:
                if pseg[1] in tg:
                    chosen_words.append(pseg[0].strip())
            except:
                print(pseg)
    #print(chosen_words)
    contents2.append(" ".join(chosen_words))    
    
"""
#print(contents2[0])

#train_df.head()

  0%|          | 66/30000 [00:02<19:01, 26.23it/s]


KeyboardInterrupt: 

In [3]:
# %%time
"""
from sklearn.svm import LinearSVC
# from sklearn.multiclass import OutputCodeClassifier #沒有必要，加上了 performance 沒有比不加好
# model = OutputCodeClassifier(LinearSVC()) #如果要使用的話語法如左
model = LinearSVC()
#MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, alpha=1e-4, solver='sgd', verbose=10, tol=1e-4, random_state=1, learning_rate_init=.1)
tfidf_vectorizer = TfidfVectorizer()
performance = cross_val_score(model, 
                              tfidf_vectorizer.fit_transform(contents2), 
                              encoded_lable, cv = 10, scoring = "f1_weighted") # scoring = "accuracy/f1_weighted"
# print("avg-accuracy: {0}".fomat(performance.mean()))
# 這裡的意義在於使用3萬筆的資料，用 MultinumialNB 模型，在 10 fold 的情況下跑出來的 performance 是多少
# 在上傳次數有限的狀況下，但想嘗試的模型有非常多的狀況下，可使用 cross validation 的方式來測試模型的 performance，來挑選最好的模型
performance.mean()
"""
# 對載入後的 training pickle 對模型進行訓練
from sklearn.naive_bayes import ComplementNB # ComplementNB/MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
model = ComplementNB() # ComplementNB/MultinomialNB
tfidf_vectorizer = TfidfVectorizer()
model.fit(tfidf_vectorizer.fit_transform(contents2), 
         encoded_lable)

ValueError: Found input variables with inconsistent numbers of samples: [66, 30000]

##### performance 紀錄
1. naive_bayes\MultinomialNB performance 0.8812004787509352，上傳結果 0.77666
2. naive_bayes\ComplementNB performance 0.8806350543597805，上傳結果 0.79916(目前最高分)
3. neighbors\KNeighborsClassifier performance 0.8451061561121213，上傳結果 0.72443
4. svm\SVC performance  ==> 這是一個不好的模型，performance 跑很久還沒有出來(30 分鐘以上)
5. naive_bayes\BernoulliNB performance 0.8752677976884107，上傳結果 0.76516
6. svm\LinearSVC performance 0.8952212892311409，上傳結果 0.761
7. neighbors\KNeighborsClassifier(n_neighbors=13) performance 0.8548916252767705，上傳結果
8. neural_network\MLPClassifierMLPClassifier(solver='lbfgs', alpha=1e-5, 
                       hidden_layer_sizes=(5, 2), random_state=1), 
                       performance 0.04829319735073786，
                       上傳結果
9. svm\LinearSVC\multiclass\OneVsRestClassifier(LinearSVC()) performance 0.8952212892311409(目前最好), 上傳結果?
10. naive_bayes\MultinomialNB\OneVsRestClassifier(ComplementNB()) performance 0.8826434815674652, 上傳結果
11. naive_bayes\MultinomialNB\OneVsRestClassifier(MultinomialNB()) performance 0.8826144826918835, 上傳結果
12. linear_model\SGDClassifier\multiclass\SGDClassifier performance 0.8927084462661558(目前三好), 上傳結果?
13. svm\LinearSVC\multiclass\OneVsOneClassifier performance 0.8946632202868514(目前次好), 上傳結果?
14. svm\LinearSVC\multiclass\OutputCodeClassifier performance 0.8872129438897028(目前四好), 上傳結果?

In [ ]:
# 將 test data 的 pickle 檔載入
test_df = pickle.load(open("./monpa_test.pickle", "rb"))
test_id = list(test_df["ID"])
test_contents = list(test_df["Content"])
test_df[3]

ID                                            Content
0   1  基督城 待 重建     外交部 籲 勿 前往     （ 中央社 記者 曾依璇 台北 24...
1   2  成龍 溼地 裝置 藝術   貼近 土地 生態 ( 影音 )       台灣 西南 沿海 地...
2   3  尼克 教頭 確認  豪哥 本 季 報銷   林書豪 到底 會不會 提早 復出 為 尼克隊 上...
3   4  台灣 智庫 新書 質疑   馬強 推 ECFA 有 不能 說 的 秘密      〔 記者 ...
4   5  宋批 台灣 共識   挑釁 會 引共憤  （ 中央社 記者 卞金峰 桃園縣 1 日 電 ） ...

: 

: 

: 

In [37]:
def data_gen(cts):
    tg=['Dfa','Dfb','Nv','VA','VAC','VB','VC','VCL','VD','VE','VF','VG','VH','VHC','VI','VJ','VK','VL','Na','A','Nc','Ncd','ORG','PER']
    result=[]
    for ct in tqdm(cts):
        chosen_words=[]
        
        psegs=mp.pseg_batch(ct)[0]
        for pseg in psegs:
            try:
                if pseg[1] in tg:
                    chosen_words.append(pseg[0].strip())
            except:
                print(pseg)
        #print(chosen_words)
        result.append(" ".join(chosen_words))
    print(result)
    return(result)

test_contents1 = data_gen(test_contents)



100%|██████████| 20000/20000 [12:53<00:00, 25.86it/s] 


['重建 外交部  籲  前往  中央社 記者 曾依璇 電 外交部 表示  強震 市中心 無 供水  交通  管制 中 民眾 內  不宜 前往 商旅   規模 強震 襲擊   造成 嚴重  傷亡   郵政 系統  暫停  服務  電力 恢', '成龍 溼地 裝置 藝術 貼近 土地 生態 影音  沿海 地區 養殖業 長期 抽取 地 下水  導致 嚴重 地層 下陷    韋恩  颱風 造成  當地  海水 倒灌   積水   退  成為 溼地   來自 國外 藝術家   利用  當地  素材  地方 志工  學生', '教頭  確認 豪哥 季 報銷 林書豪 提早 復出  尼克隊 上 場 打 熱火 季後賽   吊足 觀眾  胃口 尼克隊 總教頭 伍森 接受 東森 駐  記者   訪問 時  確定 地 說  林書豪   來不及  復出   ESPN 專欄 作家    指出    高層  很 心急  戰 豪小子  出戰 機會 高達', '智庫 新書 質疑 馬強 推 ECFA  說 秘密 記者 蘇永耀 報導 緊盯  政府 急 推 ＥＣＦＡ 問題 智庫  發表 ＥＣＦＡ  說  秘密  新 書 提出  議題 質問 馬 政府   講  利益     願  公開 代價  傷害   提 議題 質問  ', '共識 挑釁   引共憤 中央社 記者 卞金峰 電 親民黨 總統 候選人 宋楚瑜 呼籲 民眾 放下 藍綠 民生 擺 說 民進黨 總統 候選人 蔡英文  共識  狹隘   挑釁  地 做   引起    國立體育大學 體育館  舉行 中華民族 聯合 祭祖 大典 成千 上萬 信', '依 恩 法師 街頭 送   為 法寶 節  臘 月 初8 住持 依 恩 法師 右 國際 佛光會 首爾協會 會長 徐美蓉   捧 街頭 展開  送 暖 活動 中央社 記者 姜遠珍 首爾傳真 ', '羅志明 春酒 明席 開 桌 記者 侯承旭 報導 立委 羅志明 舉辦 春酒 開 桌 重回 政壇 羅志明 說  隨緣  政壇 沉寂 羅志明 舉辦 春酒宴  廣發 請 帖 包括 國民黨 秘書長 金溥聰 受  邀 國民黨  積極', '華航 加入 天合聯盟 波音 彩繪機 吸睛  馬英九 總統 見證  中華航空 正式 加入 天合聯盟 SKYTEAM 成為  會員國  提供 旅客 全球 國 航點 服務 華航 展示 波音 型 彩

In [38]:
# 對載入後的 test pickle 以訓練過之後的模型進行預測
pred = model.predict(
    tfidf_vectorizer.transform(test_contents))
pred_label = list(le.inverse_transform(pred))
#將標籤序列加入預測資料之後
test_df.insert(len(test_df.columns), "Label", pred_label)
# 刪除不要的欄位並產生結果檔
test_df = test_df.drop(["Content"], axis = 1)
test_df.to_csv("submission.csv", index = False)

ValueError: cannot insert Label, already exists